<a href="https://colab.research.google.com/github/NihaarikaAgarwal/TRIM-QA/blob/main/TAPAS_reranking_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import json
import numpy as np
from tqdm import tqdm
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

In [ ]:
#Configurations
#csv_file = "/content/sample_data/query1_test_reranking.csv"
folder_path = ""
jsonl_file = "/content/sample_data/test_reranking.jsonl"
#query = "when did season 3 of orange is the new black come out"
alpha = 1  # std dev penalty weight

In [ ]:
# Load TAPAS Model
model_name = "google/tapas-large-finetuned-wtq"
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

In [ ]:
def get_query_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    query = df.iloc[1, 0]
    return query

In [ ]:
# Load table IDs from CSV (2nd column)
def load_relevant_table_ids(csv_file):
    table_ids = set()
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                table_ids.add(row[1].strip())
    return table_ids

In [ ]:
# Load matching tables from JSONL
def load_matching_tables(jsonl_file, relevant_ids):
    matched = []
    with open(jsonl_file, 'r') as f:
        for line in f:
            table = json.loads(line)
            if table.get("tableId", "").strip() in relevant_ids:
                matched.append(table)
    return matched

In [ ]:
# Convert table format to TAPAS-compatible
def convert_to_tapas_format(table):
    headers = [col['text'] for col in table['columns']]
    data_rows = []
    for row in table['rows']:
        row_cells = [cell['text'] for cell in row['cells']]
        data_rows.append(row_cells)
    return headers, data_rows

In [ ]:
# def get_logits_from_tapas_topk(headers, rows, query):
#     df = pd.DataFrame(rows, columns=headers)

#     #tokenize
#     inputs = tokenizer(table=df, queries=[query], return_tensors="pt", truncation=True)
#     for k, v in inputs.items():
#       print("input tensor shape")
#       print(f"{k}: {v.shape}")

#     outputs = model(**inputs)
#     # Shape: (batch_size=1, seq_len)
#     logits = outputs.logits
#     #print(f"\nlogits shape: {logits.shape}")
#     #print("All logits:", logits[0].tolist())

#     # Masking logic

#     # token_type_id shape: (channel,seq_len,7)
#     token_type_ids = inputs["token_type_ids"][0]
#     # shape: (1, seq_len)
#     segment_ids = inputs["token_type_ids"][:, :, 0]
#     print(f"segment_ids shape: {segment_ids.shape}")

#     # segment_ids == 1 to get table tokens
#     cell_mask = (segment_ids[0] == 1) & (inputs["attention_mask"][0] == 1)
#     #print(f"cell_mask shape: {cell_mask.shape}")
#     valid_logits = logits[0][cell_mask]
#     #print("Valid (masked) logits:", valid_logits.tolist())

#     # top logits
#     np_logits = valid_logits.detach().cpu().numpy()
#     top_logits = np.sort(np_logits)[-10:]  # get top-k largest

#     #check
#     # tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
#     # segment_ids = inputs["token_type_ids"]
#     # attention_mask = inputs["attention_mask"]
#     # for i, token in enumerate(tokens):
#     #   seg_id = segment_ids[0, i].item()
#     #   attn = attention_mask[i].item()
#     #   logit = logits[0, i].item()
#     #   print(f"{i:>3}: {token:<15} | seg={seg_id} | mask={attn} | logit={logit:.2f}")

#     # print(f"\nNumber of tokens: {logits.shape[1]}")
#     # print(f"token_type_ids: {token_type_ids.tolist()}")
#     # print(f"attention_mask: {attention_mask.tolist()}")
#     # print(f"cell_mask: {cell_mask.tolist()}")
#     # print(f"Valid cell count: {cell_mask.sum().item()}")

#     #return valid_logits
#     return top_logits

In [ ]:
def get_logits_from_tapas(headers, rows, query):
    df = pd.DataFrame(rows, columns=headers)

    #tokenize
    inputs = tokenizer(table=df, queries=[query], return_tensors="pt", truncation=True)
    #for k, v in inputs.items():
      #print("input tensor shape")
      #print(f"{k}: {v.shape}")

    outputs = model(**inputs)
    # Shape: (batch_size=1, seq_len)
    logits = outputs.logits
    #print(f"\nlogits shape: {logits.shape}")
    #print("All logits:", logits[0].tolist())

    # Masking logic

    # token_type_id shape: (channel,seq_len,7)
    token_type_ids = inputs["token_type_ids"][0]
    # shape: (1, seq_len)
    segment_ids = inputs["token_type_ids"][:, :, 0]
    #print(f"segment_ids shape: {segment_ids.shape}")

    # segment_ids == 1 to get table tokens
    cell_mask = (segment_ids[0] == 1) & (inputs["attention_mask"][0] == 1)
    #print(f"cell_mask shape: {cell_mask.shape}")
    valid_logits = logits[0][cell_mask]
    #print("Valid (masked) logits:", valid_logits.tolist())

    # top logits
    #np_logits = valid_logits.detach().cpu().numpy()
    #top_logits = np.sort(np_logits)[-10:]  # get top-k largest

    #check
    # tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    # segment_ids = inputs["token_type_ids"]
    # attention_mask = inputs["attention_mask"]
    # for i, token in enumerate(tokens):
    #   seg_id = segment_ids[0, i].item()
    #   attn = attention_mask[i].item()
    #   logit = logits[0, i].item()
    #   print(f"{i:>3}: {token:<15} | seg={seg_id} | mask={attn} | logit={logit:.2f}")

    # print(f"\nNumber of tokens: {logits.shape[1]}")
    # print(f"token_type_ids: {token_type_ids.tolist()}")
    # print(f"attention_mask: {attention_mask.tolist()}")
    # print(f"cell_mask: {cell_mask.tolist()}")
    # print(f"Valid cell count: {cell_mask.sum().item()}")

    return valid_logits
    #return top_logits

In [ ]:
# def get_logits_from_tapas_clip(headers, rows, query, clip_min=-100, clip_max=100):
#     df = pd.DataFrame(rows, columns=headers)

#     #tokenize
#     inputs = tokenizer(table=df, queries=[query], return_tensors="pt", truncation=True)
#     # for k, v in inputs.items():
#     #   print("input tensor shape")
#     #   print(f"{k}: {v.shape}")

#     outputs = model(**inputs)
#     # Shape: (batch_size=1, seq_len)
#     logits = outputs.logits
#     print(f"\nlogits shape: {logits.shape}")
#     print("All logits:", logits[0].tolist())

#     # Masking logic

#     # token_type_id shape: (channel,seq_len,7)
#     token_type_ids = inputs["token_type_ids"][0]
#     # shape: (1, seq_len)
#     segment_ids = inputs["token_type_ids"][:, :, 0]
#     print(f"segment_ids shape: {segment_ids.shape}")

#     # segment_ids == 1 to get table tokens
#     cell_mask = (segment_ids[0] == 1) & (inputs["attention_mask"][0] == 1)
#     print(f"cell_mask shape: {cell_mask.shape}")
#     valid_logits = logits[0][cell_mask]
#     print("Valid (masked) logits:", valid_logits.tolist())

#     # top logits
#     np_logits = valid_logits.detach().cpu().numpy()
#     #top_logits = np.sort(np_logits)[-10:]  # get top-k largest

#     #clipped logits
#     clipped_logits = np.clip(np_logits, clip_min, clip_max)

#     #check
#     # tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
#     # segment_ids = inputs["token_type_ids"]
#     # attention_mask = inputs["attention_mask"]
#     # for i, token in enumerate(tokens):
#     #   seg_id = segment_ids[0, i].item()
#     #   attn = attention_mask[i].item()
#     #   logit = logits[0, i].item()
#     #   print(f"{i:>3}: {token:<15} | seg={seg_id} | mask={attn} | logit={logit:.2f}")

#     # print(f"\nNumber of tokens: {logits.shape[1]}")
#     # print(f"token_type_ids: {token_type_ids.tolist()}")
#     # print(f"attention_mask: {attention_mask.tolist()}")
#     # print(f"cell_mask: {cell_mask.tolist()}")
#     # print(f"Valid cell count: {cell_mask.sum().item()}")

#     return clipped_logits
#     #return top_logits


In [ ]:
# # Compute mean, std, and score
# def score_logits_topk(valid_logits, alpha=1.0):
#   #if valid_logits.numel() > 0:
#   if len(valid_logits) > 0:
#     mean = valid_logits.mean().item()
#     std = valid_logits.std().item()
#     score = mean - alpha * std
#     #print(f"\n Valid logits shape: {valid_logits.shape}")
#     print(f"Mean: {mean:.3f}, Std: {std:.3f}, Score: {score:.3f}")
#     return score, mean, std
#   else:
#     print("No valid table cell logits found after masking!")
#     return float('-inf'), 0.0, 0.0

In [ ]:
# Compute mean, std, and score
def score_logits(valid_logits, alpha=1.0):
  if valid_logits.numel() > 0:
  #if len(valid_logits) > 0:
    mean = valid_logits.mean().item()
    std = valid_logits.std().item()
    score = mean - alpha * std
    #print(f"\n Valid logits shape: {valid_logits.shape}")
    print(f"Mean: {mean:.3f}, Std: {std:.3f}, Score: {score:.3f}")
    return score, mean, std
  else:
    print("No valid table cell logits found after masking!")
    return float('-inf'), 0.0, 0.0

In [ ]:
# import numpy as np

# def score_logits_clip(valid_logits, alpha=1.0, clip_min =-100, clip_max=100):

#     # Convert to numpy array safely
#     if hasattr(valid_logits, "detach"):
#         valid_logits = valid_logits.detach().cpu().numpy()
#     else:
#         valid_logits = np.array(valid_logits)

#     # Check if there are any valid logits
#     if len(valid_logits) > 0:
#         clipped_logits = np.clip(valid_logits, clip_min, clip_max)

#         mean = np.mean(clipped_logits)
#         std = np.std(clipped_logits)
#         score = mean - alpha * std

#         print(f"Clipped Mean: {mean:.3f}, Std: {std:.3f}, Score: {score:.3f}")
#         return score, mean, std
#     else:
#         print(" No valid table cell logits found after masking!")
#         return float('-inf'), 0.0, 0.0


In [ ]:
# Main pipeline
def rank_relevant_tables(csv_file):
    query = get_query_from_csv(csv_file)
    relevant_ids = load_relevant_table_ids(csv_file)
    matched_tables = load_matching_tables(jsonl_file, relevant_ids)
    print(f"Loaded {len(matched_tables)} tables matching IDs.")

    ranked = []
    for table in tqdm(matched_tables, desc="Scoring tables"):
        headers, rows = convert_to_tapas_format(table)
        for i, row in enumerate(rows):
          if len(row) != len(headers):
              print(f"[Mismatch] Table ID: {table.get('tableId')}")
              print(f"Header length: {len(headers)}")
              print(f"Row {i} length: {len(row)} → Row content: {row}")

        # print("Headers:", headers)
        # print("First row:", rows[0] if rows else "No rows")
        # print("Query:", query)

        try:
            logits = get_logits_from_tapas(headers, rows, query)
            #logits = get_logits_from_tapas_topk(headers, rows, query)
            #logits = get_logits_from_tapas_clip(headers, rows, query, clip_min=-100, clip_max=100)
            score, mean, std = score_logits(logits, alpha)
            #score, mean, std = score_logits_topk(logits, alpha)
            #score, mean, std = score_logits_clip(logits, alpha, clip_min=-100, clip_max=100)
            ranked.append({
                "tableId": table["tableId"],
                "score": score,
                "mean": mean,
                "std": std
            })
        except Exception as e:
            print(f"Failed to process table {table['tableId']}: {e}")

    ranked_sorted = sorted(ranked, key=lambda x: x["score"], reverse=True)

    # Print top 10
    # for i, entry in enumerate(ranked_sorted[:10]):
    #     print(f"{i+1}. {entry['tableId']} | Score: {entry['score']:.3f} | Mean: {entry['mean']:.3f} | Std: {entry['std']:.3f}")

    return ranked_sorted, query

In [ ]:
def calculate_recall_from_csv(ranked_tables):

    ranked_table_ids = [entry["tableId"] for entry in ranked_tables]

    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        rows = list(reader)
        if len(rows) < 2 or len(rows[1]) < 3:
            #print("CSV does not have the target table.")
            return
        target_id = rows[1][2].strip()

    top_10_cutoff = max(1, int(0.10 * len(ranked_table_ids)))
    top_20_cutoff = max(1, int(0.20 * len(ranked_table_ids)))

    if target_id in ranked_table_ids:
        rank = ranked_table_ids.index(target_id) + 1
        in_10 = int(rank <= top_10_cutoff)
        in_20 = int(rank <= top_20_cutoff)
        # print(f"Target table {target_id} found at rank {rank}")
        # print(f"Recall@10%: {in_10}, Recall@20%: {in_20}")
        return target_id,rank,in_10, in_20
    else:
        #print(f" Target table {target_id} not found in reranked list.")
        return target_id, -1, 0, 0

In [ ]:
output_rows = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        csv_file = os.path.join(folder_path, file_name)
        #print(f"Processing CSV file: {csv_file}")
        ranked_sorted, query = rank_relevant_tables(csv_file)
        target_id,rank, in_10, in_20 = calculate_recall_from_csv(ranked_sorted)
        output_rows.append([query, target_id, rank, in_10, in_20])

# Write the results to a CSV
output_csv = "ranked_results.csv"
header = ["Query", "Target table","Target Table Rank", "Recall at 20%", "Recall at 10%"]

with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(output_rows)

print(f"Results written to {output_csv}")

In [ ]:
# Run
#clip, 1
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()

Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]

input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:10<03:10, 10.58s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:35<00:04,  4.17s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:41<00:00,  5.32s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
recall = calculate_recall_from_csv(ranked_tables)


✅ Target table List of Orange Is the New Black episodes_ED31ED6C18A3E20B found at rank 3
Recall@10%: 0, Recall@20%: 1


In [ ]:
# Run
#clip, 0.8
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:10<03:13, 10.73s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [02:22<00:05,  5.45s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [02:29<00:00,  7.88s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#clip, 1.2
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:06<01:57,  6.53s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:44<00:04,  4.96s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:52<00:00,  5.90s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#topk, 1
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]

input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:06<01:54,  6.34s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:39<00:04,  4.70s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:47<00:00,  5.66s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#topk, 0.8
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:06<01:58,  6.58s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:38<00:04,  4.57s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:46<00:00,  5.59s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#topk, 1.2
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:06<01:57,  6.55s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:44<00:04,  5.00s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:52<00:00,  5.93s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#ususal, 1
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]

input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:07<02:07,  7.11s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:39<00:04,  4.60s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:46<00:00,  5.63s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#usual, 0.8
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:05<01:40,  5.59s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:34<00:04,  4.36s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:41<00:00,  5.35s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#usual, 1.2
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()


Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:06<01:53,  6.29s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:33<00:04,  4.34s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:41<00:00,  5.33s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.

In [ ]:
# Run
#clip, 0.5
if __name__ == "__main__":
    ranked_tables = rank_relevant_tables()

Loaded 19 tables matching IDs.


Scoring tables:   0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


input tensor shape
input_ids: torch.Size([1, 434])
input tensor shape
token_type_ids: torch.Size([1, 434, 7])
input tensor shape
attention_mask: torch.Size([1, 434])


Scoring tables:   5%|▌         | 1/19 [00:07<02:15,  7.51s/it]


logits shape: torch.Size([1, 434])
All logits: [-10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10029.3857421875, -10040.2607421875, -10040.2607421875, -10040.2607421875, -10034.92578125, -10034.92578125, -10034.92578125, -10044.611328125, -10044.611328125, -10044.611328125, -80.61176300048828, -80.61176300048828, -80.61176300048828, -10033.9755859375, -10033.9755859375, -10033.9755859375, -10042.791015625, -10036.9169921875, -10046.1171875, -10046.1171875, -10046.1171875, -41.916603088378906, -41.916603088378906, -41.916603088378906, -10035.8447265625, -10035.8447265625, -10035.8447265625, -10044.60546875, -10044.60546875, -10044.60546875, -10035.75, -10035.75, -10035.75, -10039.599609375, -10039.599609375, -10039.599609375, -52.412078857421875, -52.412078857421875, -52.412078857421875, 

...


logits shape: torch.Size([1, 63])
All logits: [-10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10018.146484375, -10050.2685546875, -10032.720703125, -10032.720703125, -10032.720703125, -10032.1181640625, -10030.44921875, -10030.44921875, -10030.44921875, -131.6600341796875, -131.6600341796875, -10040.1796875, -10040.1796875, -10040.1796875, -10040.1796875, -10025.6376953125, -10025.8935546875, -10025.1181640625, -29.20894432067871, -29.20894432067871, -29.20894432067871, -29.20894432067871, -10024.1796875, -10025.4501953125, -10025.9150390625, -18.18457794189453, -18.18457794189453, -18.18457794189453, -18.18457794189453, -10013.2724609375, -10013.2470703125, -10013.6142578125, 19.468204498291016, 19.468204498291016, 19.468204498291016, 19.468204498291016, -10022.99609375, -10023.669921875, -10022.659

Scoring tables:  95%|█████████▍| 18/19 [01:35<00:04,  4.51s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -10010.84765625, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -47.73303985595703, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -10034.9150390625, -30.927322387695312, -10020.3671875, -10020.3671875

Scoring tables: 100%|██████████| 19/19 [01:43<00:00,  5.43s/it]


logits shape: torch.Size([1, 511])
All logits: [-10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -10010.427734375, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -56.70947265625, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -10050.6328125, -33.27557373046875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.6171875, -10022.